In [11]:
import numpy
import numpy as np
import torch
import torch.nn as nn 
import torch.optim as optim
import random
import pandas

In [46]:
#data_generator
matrix_T = torch.tensor([[0, 1], [1, 0]], dtype=torch.float64)
matrix_U = torch.tensor([[1, -1], [0, 1]], dtype=torch.float64)

inverse_T = torch.inverse(matrix_T)
inverse_U = torch.inverse(matrix_U)

max_moves = 30  #how many inverse operations on the final destination
goal_position = torch.tensor([[1], [0]], dtype=torch.float64)


def data_generator(num_of_datapoints):
    data = torch.empty((num_of_datapoints, 3), dtype=torch.float64)
    for i in range(num_of_datapoints):
        start = goal_position
        moves = random.randint(1, max_moves)
        coin = 3
        for q in range(moves):
            coin = random.randint(0, 1)
            if coin == 1:
                #  print(start, inverse_U, "=")
                start = torch.matmul(inverse_U, start)
            #  print(start)
            else:
                #  print(start, inverse_T, "=")
                start = torch.matmul(inverse_T, start)
            # print(start)
        # print("done")
        data[i, 0] = start[0]
        data[i, 1] = start[1]
        data[i, 2] = coin
    return data

#https://stackoverflow.com/questions/36158058/torch-save-tensor-to-csv-file#:~:text=For%20simple%20tables%2C%20you%20can,then%20to%20a%20Pandas%20dataframe.&text=You%20can%20first%20convert%20the,table%20as%20a%20csv%20file.
data = data_generator(100)
data_table = data.numpy()
df = pandas.DataFrame(data_table)
df.to_csv("generated_pairs.csv", index = False) 


In [47]:
#https://machinelearningmastery.com/develop-your-first-neural-network-with-pytorch-step-by-step/
#https://stackoverflow.com/questions/12336234/read-csv-file-to-numpy-array-first-row-as-strings-rest-as-float
#https://www.w3schools.com/python/numpy/numpy_array_join.asp#:~:text=Joining%20Arrays%20Using%20Stack%20Functions&text=We%20can%20concatenate%20two%201,it%20is%20taken%20as%200.

#make tensors from csv file
dataset = np.genfromtxt("generated_pairs.csv", dtype = float, names=True, delimiter=",")
a = dataset['0'] #arry of all the a's
b = dataset['1'] #array of all the b's
pairs = np.stack((a,b),axis=1)
x = torch.tensor(pairs, dtype = torch.float64) #input to a tensor
first_move = dataset['2'] #array of all the first moves (0 or 1)
y = torch.tensor(first_move, dtype = torch.float64) #output to a tensor 


In [48]:
#https://machinelearningmastery.com/develop-your-first-neural-network-with-pytorch-step-by-step/
#making the model 
model = nn.Sequential(
    nn.Linear(2, 128), #first layer 128
    nn.ReLU(),
    nn.Linear(128,64), #second layer 64
    nn.ReLU(),
    nn.Linear(64, 16), #third layer 16
    nn.ReLU(),
    nn.Linear(16,1), #singular output 
    nn.Softmax()
)
#print(model)

In [49]:
#https://machinelearningmastery.com/develop-your-first-neural-network-with-pytorch-step-by-step/
#prep for training 
loss_function = nn.BCELoss() #should this be our loss function too? 
optimizer = optim.Adam(model.parameters(), lr = 0.001); #chose Adam and lr from site 

#split training and testing data 
train, test = torch.utils.data.random_split(x, [0.75, 0.25]) #from cs544 class notes 
#I know that I am going to run into problems here because I split the input and output so somehow I have to recover the split in order to have the expected results 

In [54]:
#https://machinelearningmastery.com/develop-your-first-neural-network-with-pytorch-step-by-step/
train_rows, train_col = train.get_shape() 
num_epochs = train_rows
batch_size = num_epochs / 10  #this is going to run into problems b/c the last one won't have 10 
for epoch in range(n_epochs):
    for i in range(0, len(train), batch_size):
        train_batch = train[i:i+batch_size]
        y_pred = model(train_batch)
        y_batch = y[i:i+batch_size]
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

AttributeError: 'Subset' object has no attribute 'get_shape'